In [64]:
import os 
import torch

base_dir = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train/SURREAL_128_1-1-2-2ev_template_remeshed_augShapes/train'



In [65]:
# for each .pt file in the directory, read it and check for nan values
for file in os.listdir(base_dir):
    if file.endswith('.pt'):
        print(file)
        data = torch.load(os.path.join(base_dir, file), mmap=True)
        # print(data)
        print(torch.isnan(data).any())
        print(torch.isinf(data).any())
        print(data.size())
        
        print(data.min(), data.max())
        
        print('\n')

In [66]:
# get C_gt_xy
C_gt_xy = torch.load('/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train/SURREAL_128_1-1-2-2ev_template_remeshed_augShapes/train/C_gt_xy.pt')

In [69]:
# get the index of the minimum value in C_gt_xy, shape is [230000, 32, 32], without flattening
idx_min = torch.argmin(C_gt_xy)

# the unflattened version
idx_min_unflattened = torch.unravel_index(idx_min, C_gt_xy.size())
print(idx_min_unflattened, idx_min)

In [71]:
import my_code.utils.plotting_utils as plotting_utils
import matplotlib.pyplot as plt
import torch

l = 0
h = 64

fig, axs = plt.subplots(1, 2, figsize=(15, 4))

plotting_utils.plot_Cxy(fig, axs[0],  C_gt_xy[idx_min_unflattened[0]],
                        'before', l, h, show_grid=False, show_colorbar=False)
plt.show()

In [77]:
# get a sample optimizer
optimizer = torch.optim.Adam([torch.tensor([1.0])], lr=1e-3)

cosine_lr = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000, eta_min=0, last_epoch=-1)

In [91]:
from diffusers.optimization import get_cosine_schedule_with_warmup

cosine_lr = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=1000,
)

In [92]:
# plot the learning rate
import matplotlib.pyplot as plt

lrs = []
for epoch in range(1000):
    cosine_lr.step()
    lrs.append(optimizer.param_groups[0]['lr'])
    
plt.plot(lrs)
plt.show()

In [93]:
# get the size of this file in mega bytes: /home/s94zalek_hpc/shape_matching/data/SHREC19_r/diffusion/0f616c57e8c496fcdce20f60a6224bc6dc7256ac_0.npz

import os

file_path = '/home/s94zalek_hpc/shape_matching/data/SHREC19_r/diffusion/0f616c57e8c496fcdce20f60a6224bc6dc7256ac_0.npz'

file_size = os.path.getsize(file_path) / (1024 * 1024)
print(file_size)